In [ ]:
!pip install tensorflow_text natasha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 54.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=a1f399653f3f9fb057cc6403b75d3ab8c267c359b778d539d80a0060f69d3d0f
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26094 sha256=f2d803f0d051479b851dd9dffa149fdabae2c5308bf1555fc29babdcae0c9d10
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built docopt intervaltree


In [ ]:
import re
import time

import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup  # Для удаления HTML тегов

import tensorflow as tf
from tensorflow.keras.saving import load_model
import tensorflow_hub as hub
import tensorflow_text as text

from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,
    Doc
)
from google.colab import drive
from matplotlib import pyplot as plt
import joblib

# Dataset_import

In [ ]:
# вход на файл
drive.mount("/content/drive")
DIR = '/content/drive/My Drive/Colab Notebooks/'
df = pd.read_excel(DIR + "CRA_train_1200.xlsx")[:10]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Models loading
<b style="color:red"> warning! it cat take a lot of time! </b>

In [ ]:
model_cat = load_model(DIR + 'model_cat_2.keras', custom_objects={'KerasLayer': hub.KerasLayer})
model_rat = load_model(DIR + 'model_rat_2.keras', custom_objects={'KerasLayer': hub.KerasLayer})

In [ ]:
le_cat = joblib.load(DIR + 'label_encoder_7.joblib')
le_rat = joblib.load(DIR + 'label_encoder_17.joblib')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Inicialize class

In [ ]:
class Nlp:
    def __init__(self):
        # Natasha
        self.segmenter = Segmenter()
        self.morph_vocab = MorphVocab()
        self.emb = NewsEmbedding()
        self.morph_tagger = NewsMorphTagger(self.emb)
        self.syntax_parser = NewsSyntaxParser(self.emb)
        self.ner_tagger = NewsNERTagger(self.emb)
        self.names_extractor = NamesExtractor(self.morph_vocab)
        self.dates_extractor = DatesExtractor(self.morph_vocab)
        self.money_extractor = MoneyExtractor(self.morph_vocab)
        self.addr_extractor = AddrExtractor(self.morph_vocab)


    # Очищаем текст регулярными выражениями
    def clear_text(self, text):
        soup = BeautifulSoup(text)
        text = soup.get_text()
        text = re.sub(r'(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)', '', text)
        text = re.sub(r'[\n\t]', ' ', text).strip()  # Перенос, табуляция
        text = re.sub(r'[^\w\d\s\.\,]', ' ', text)  # Только слова, цифры, пробелы, точки и запятые
        text = re.sub(r'\s+', ' ', text)  # Удаляем двойные пробелы
        return text


    # Извлекаем фичи
    def get_features(self, df):
        start_time = time.time()
        features_list = []
        for i in range(df.shape[0]):
            ner_list = self.get_ner_features(df.iloc[i])
            words = df.iloc[i].split()
            count = len(words)  # Количество слов в строке
            if count > 0:
                average = sum(len(word) for word in words) / count
                uniq = round(100*len(set(words))/count)  # % уникальных слов в строке
                features_list.append(ner_list + [count, average, uniq])
            if (i+1) % 10 == 0:
                delta_time = round(time.time() - start_time)
                print(f'Обработано {i+1} из {df.shape[0]}, {delta_time}c')
        return np.array(features_list)


    # В категорийные признаки
    def to_categorical(self, df, labels):
        le = LabelEncoder()
        le.fit(labels)
        label = le.transform(df)
        return to_categorical(label, num_classes=len(labels), dtype='int')


    # Возвращает список с количеством найденных именованных сущностей [names, dates, LOC, MONEY]
    def get_ner_features(self, text):
        names = len(list(nlp.names_extractor(text)))
        money = len(list(nlp.money_extractor(text)))
        addr = len(list(nlp.addr_extractor(text)))
        return [names, money, addr]


    # Извлекает именнованные сущности - имена, названия
    def names_extractor(self, text):
        return self.names_extractor(text)


    # Извлекает именнованные сущности - даты
    def dates_extractor(self, text):
        return self.dates_extractor(text)


    # Извлекает именнованные сущности - деньги
    def money_extractor(self, text):
        return self.money_extractor(text)


    # Извлекает именнованные сущности - локацию, адреса
    def addr_extractor(self, text):
        return self.addr_extractor(text)

    def prediction_pipeline(self, df):
      copied_df = df["pr_txt"].copy()
      cleared_text_list = list(copied_df.map(self.clear_text))
      df_text = pd.DataFrame(cleared_text_list, columns=['text'])
      x_features = self.get_features(df_text['text'])
      df_features = pd.DataFrame(x_features, columns=['name', 'date', 'location', 'count', 'average', 'uniq'])
      pred_cat_v = model_cat.predict([df_text, df_features])
      pred_rat_v = model_rat.predict([df_text, df_features])
      pred_cat_am = np.argmax(pred_cat_v, axis=1)
      pred_rat_am = np.argmax(pred_rat_v, axis=1)
      pred_cat_y = le_cat.inverse_transform(pred_cat_am)
      pred_rat_y = le_rat.inverse_transform(pred_rat_am)
      answer = pd.DataFrame({'Категория': pred_cat_y, 'Уровень рейтинга': pred_rat_y})
      out = pd.concat([copied_df, answer], axis=1)
      out.to_excel('answer.xlsx', index=False)


In [ ]:
nlp = Nlp()

In [ ]:
nlp.prediction_pipeline(df)

Обработано 10 из 10, 81c
1/1 [==============================] - 6s 6s/step
0    Повышение кредитного рейтинга  Акционерного об...
1    «Эксперт РА» подтвердил кредитный рейтинг комп...
2    НКР повысило кредитный рейтинг ООО "ОТЭКО-Порт...
3    «Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...
4    29 марта 2023 г. Ведущий рейтинговый аналитик ...
5    Кредитный рейтинг  ПАО «ФосАгро» (далее — Комп...
6    «Эксперт РА» повысил кредитный рейтинг ОАО «МР...
7    «Эксперт РА» понизил кредитный рейтинг ПАО «М....
8    «Эксперт РА» повысил кредитный рейтинг компани...
9    Кредитный рейтинг  ООО «МВМ»  (далее — Компани...
Name: pr_txt, dtype: object
  Категория Уровень рейтинга
0         A                A
1        BB               BB
2         A                A
3       AAA              AAA
4       BBB              BBB
5       AAA              AAA
6        AA              AA+
7         A                A
8        BB              BB+
9         A                A
